In [1]:
from netgen.csg import *
from ngsolve import *
from ngsolve.webgui import Draw
# from netgen import gui
from ngsolve.comp import ConvertOperator
import numpy as np

import time
import pickle
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d

import logging
from datetime import datetime

build mesh and energy density

In [2]:
%matplotlib notebook

show = False
with TaskManager():
    geo = CSGeometry()
    box_size = 2
    box = OrthoBrick(Pnt(-box_size,-box_size,-box_size), Pnt(box_size,box_size,box_size)).bc("outer")

    lx = 0.4
    lz = 0.5
    d = 0.1
    lp = 0.3
    core = OrthoBrick(Pnt(-lx,-d/2,-lz),Pnt(lx,d/2,lz))- \
               OrthoBrick(Pnt(d-lx,-d,-lz+d),Pnt(lx-d,d,lz-d))- \
               OrthoBrick(Pnt(d,-d,-d/2),Pnt(lx+d,d,d/2)).maxh(0.2).mat('core')

    plate = OrthoBrick(Pnt(d/2,-lp,-d/4),Pnt(2*lx-d,lp,d/4)).maxh(0.05).mat('plate')


    coil = (Cylinder(Pnt(-lx+d/2,0,0), Pnt(-lx+d/2,0,1), 3*d) - \
            Cylinder(Pnt(-lx+d/2,0,0), Pnt(-lx+d/2,0,1), d)) * \
            OrthoBrick (Pnt(-1,-1,-lz+2*d),Pnt(1,1,lz-2*d)).maxh(0.2).mat('coil')

    air = (box - coil - plate -core).mat('air') 

    geo.Add(plate)
    geo.Add(core)
    geo.Add(coil)
    geo.Add(air, transparent=True)

    mesh = Mesh(geo.GenerateMesh(maxh=0.5))
    mesh.Curve(3)

    H_KL = [ -4.47197834e-13, 1.60000000e+01, 3.00000000e+01, 5.40000000e+01\
    , 9.30000000e+01, 1.43000000e+02, 1.91000000e+02, 2.10000000e+02 \
    , 2.22000000e+02, 2.33000000e+02, 2.47000000e+02, 2.58000000e+02\
    , 2.72000000e+02, 2.89000000e+02, 3.13000000e+02, 3.42000000e+02\
    , 3.77000000e+02, 4.33000000e+02, 5.09000000e+02, 6.48000000e+02\
    , 9.33000000e+02, 1.22800000e+03, 1.93400000e+03, 2.91300000e+03\
    , 4.99300000e+03, 7.18900000e+03, 9.42300000e+03, 9.42300000e+03\
    , 1.28203768e+04, 1.65447489e+04, 2.07163957e+04, 2.55500961e+04\
    , 3.15206135e+04, 4.03204637e+04, 7.73038295e+04, 1.29272791e+05\
    , 1.81241752e+05, 2.33210713e+05, 2.85179674e+05, 3.37148635e+05\
    , 3.89117596e+05, 4.41086557e+05, 4.93055518e+05, 5.45024479e+05\
    , 5.96993440e+05, 6.48962401e+05, 7.00931362e+05, 7.52900323e+05\
    , 8.04869284e+05, 8.56838245e+05, 9.08807206e+05, 9.60776167e+05\
    , 1.01274513e+06, 1.06471409e+06, 1.11668305e+06, 1.16865201e+06\
    , 1.22062097e+06, 1.27258993e+06, 1.32455889e+06, 1.37652785e+06\
    , 1.42849682e+06, 1.48046578e+06, 1.53243474e+06, 1.58440370e+06\
    , 1.63637266e+06, 1.68834162e+06, 1.74031058e+06, 1.79227954e+06\
    , 1.84424850e+06, 1.89621746e+06, 1.94818643e+06, 2.00015539e+06\
    , 2.05212435e+06, 2.10409331e+06, 2.15606227e+06, 2.20803123e+06\
    , 2.26000019e+06]

    B_KL = [ 0.00000000e+00, 2.50000000e-03, 5.00000000e-03, 1.25000000e-02\
    , 2.50000000e-02, 5.00000000e-02, 1.00000000e-01, 2.00000000e-01\
    , 3.00000000e-01, 4.00000000e-01, 5.00000000e-01, 6.00000000e-01\
    , 7.00000000e-01, 8.00000000e-01, 9.00000000e-01, 1.00000000e+00\
    , 1.10000000e+00, 1.20000000e+00, 1.30000000e+00, 1.40000000e+00\
    , 1.50000000e+00, 1.55000000e+00, 1.60000000e+00, 1.65000000e+00\
    , 1.70000000e+00, 1.75000000e+00, 1.80000000e+00, 1.80000000e+00\
    , 1.86530612e+00, 1.93061224e+00, 1.99591837e+00, 2.06122449e+00\
    , 2.12653061e+00, 2.19183673e+00, 2.25714286e+00, 2.32244898e+00\
    , 2.38775510e+00, 2.45306122e+00, 2.51836735e+00, 2.58367347e+00\
    , 2.64897959e+00, 2.71428571e+00, 2.77959184e+00, 2.84489796e+00\
    , 2.91020408e+00, 2.97551020e+00, 3.04081633e+00, 3.10612245e+00\
    , 3.17142857e+00, 3.23673469e+00, 3.30204082e+00, 3.36734694e+00\
    , 3.43265306e+00, 3.49795918e+00, 3.56326531e+00, 3.62857143e+00\
    , 3.69387755e+00, 3.75918367e+00, 3.82448980e+00, 3.88979592e+00\
    , 3.95510204e+00, 4.02040816e+00, 4.08571429e+00, 4.15102041e+00\
    , 4.21632653e+00, 4.28163265e+00, 4.34693878e+00, 4.41224490e+00\
    , 4.47755102e+00, 4.54285714e+00, 4.60816327e+00, 4.67346939e+00\
    , 4.73877551e+00, 4.80408163e+00, 4.86938776e+00, 4.93469388e+00\
    , 5.00000000e+00]
    bh_curve = BSpline (2, [0]+list(B_KL), list(H_KL)) # [0] + is needed!
    energy_dens = bh_curve.Integrate()

if show: 
    Draw (mesh)

#     plt.figure(1, figsize=[12, 9])
    plt.clf()
    plt.plot(H_KL, B_KL, '.-r')
    plt.xlim(0, 2000)
    plt.ylim(0, 2.12)
    plt.grid()
    plt.xlabel("Magnetic Field Strength H in A/m")
    plt.ylabel("Magnetic Flux Density B in T")



$$
\DeclareMathOperator{\Grad}{grad}
\DeclareMathOperator{\Curl}{curl}
\DeclareMathOperator{\Div}{div}
\DeclareMathOperator{\R}{\mathbb{R}}
\DeclareMathOperator{\real}{real}
\DeclareMathOperator{\reg}{reg}
\DeclareMathOperator{\icd}{icd}
\DeclareMathOperator{\cur}{cur}
$$
### Problem description

Consider $j(x,t) = j(x)\cos(\omega t)$.


Find a $ u \in C(\R^+; H(\Curl))$ such that 

$$
\int_{\text{plate}} H(\Curl u )\Curl v + \frac{1}{\mu}\int_{\Omega\setminus \text{plate}} \Curl u \Curl v+ \reg \int_{\Omega\setminus \text{plate}} u v -\int_{\text{coil}} j v + \sigma \int_{\text{plate}} \partial_t u v =  0 \qquad  \forall \, v \in H(\Curl) \tag{1}
$$

Defining the functional


$$
\tilde F(u) := \int_{\text{plate}} W(\Curl u|) + \frac{1}{2\mu}\int_{\Omega\setminus \text{plate}} (\Curl u )^2+ \reg \int \frac{1}{2} u^2  - \int ju  
$$

(1) is equivalent to: Find a $u\in H(\Curl)$ such that 

$$
\partial_v \tilde F(u) := \lim_{\epsilon \rightarrow 0} \frac{d}{d\epsilon} \tilde F(u+\epsilon v) = -\sigma \int_{\text{plate}} \partial_t u v \qquad \forall v \in H(\Curl)
$$

where 
$$
W(x) = \int_0^{x} \frac{1}{\mu(s)}s ~ ds.
$$


Applying an implicit Euler iteration yields the equation 

$$
\partial_v \tilde F(u^{n+1}) + \sigma \int_{\text{plate}} \frac{u^{n+1} - u^{n}}{\Delta t}v = 0    \qquad  \forall \, v \in H(\Curl) \tag{2}
$$
<!-- 
$$
\int_{\text{plate}} H(\Curl u^{n+1} )\Curl v + \frac{1}{\mu}\int_{\Omega\setminus \text{plate}} \Curl u^{n+1} \Curl v+ \reg \int_{\Omega\setminus \text{plate}} u^{n+1} v -\int_{\text{coil}} j v + \\ \sigma \int_{\text{plate}} \partial_t u^{n+1} v - \sigma \int_{\text{plate}} \partial_t u^{n} v = 0 
$$
 -->
which is equivalent to minimizing the functional 

$$
F(u) = \tilde F(u)+\frac{\sigma}{2\Delta t }\int_{\text{plate}} (u-u^n)^2
$$

and setting $u^{n+1} := u$.

$F$ is convex and therefore the minimum can be found by Newton. One iteration contains the following steps

1. $r^i := \partial_{v} F (\tilde u^i)$ 
* $B^i := \partial_{\tilde v}^2 F(\tilde u ^i)$
* solve $B^i \Delta \tilde u^i = -r^i$
* update $\tilde u^{i+1} = \tilde u^i + \Delta \tilde u^i$



From now on define

$$
\begin{split}
g(u) &:= \int_{\text{plate}} w(|\Curl u|) \\
a(u,v) &:= \frac{1}{\mu}\int_{\Omega\setminus \text{plate}} \Curl u \Curl v+ \reg \int_{\Omega\setminus \text{plate}} u v \\
\icd(v) &:= -\int_{\text{coil}} j v \\
\cur(u,v) &:= \sigma \int_{\text{plate}}  u v
\end{split}
$$

Now we can write 

$$
\begin{split}
\partial_v F(u) &= \partial_v g(u) +  a(u,v) + \frac{1}{\Delta t}\cur (u,v) -\frac{1}{\Delta t}\cur (u^n,v) + \icd (v) \\
\partial_v^2 F(u) & = \partial_v^2 g(u) +  a(v,v) + \frac{1}{\Delta t}\cur (v,v)
\end{split}
$$

In [4]:
# define constants and current density function
mu0 = 4*np.pi*1e-7
mur = { "core" : 1e10, "coil" : 1, "air" : 1}#, 'plate': 1}
sigma = {"plate": 8.6*1e6}
omega = 2*np.pi*30 # angular frequency
time_interval = [0.1, 0.3]
t = Parameter(0)
j = CoefficientFunction((-y,lx-d/2+x,0))*CoefficientFunction(1000/sqrt(y**2+(lx-d/2+x)**2)*cos(t*omega))

with TaskManager():
    # define space
    fes = HCurl(mesh, order = 3, dirichlet="outer")#, nograds=True)
    u,v = fes.TnT()
    proj = Projector(fes.FreeDofs(), True)

    # define (bi-) linear froms
    g = BilinearForm (fes)
    g += Variation(energy_dens(sqrt(1e-12+ curl(u)*curl(u))) * dx('plate'))
    

    a = BilinearForm(fes)
    for mat in mur.keys():
        a += 1/(mu0*mur[mat])*curl(u)*curl(v)*dx(mat) + 1e-6*1/(mu0*mur[mat])*u*v*dx(mat)
#     a += 1e-6*u*v*dx('plate')
    a.Assemble()
    
    
    

    icd = LinearForm(fes)
    icd += -j*v*dx('coil')

    cur = BilinearForm(fes)
    cur += sigma['plate']*u*v*dx('plate')
    cur.Assemble()
    
    gfu = GridFunction(fes)
    
date = datetime.now().strftime("%d-%b-%Y (%H:%M:%S)")
logging.basicConfig(filename='output/nonlinear_{}.log'.format(date),level=logging.DEBUG)
    

Calculate reduced basis

In [ ]:
# date = datetime.now().strftime("%d-%b-%Y (%H:%M:%S)")
# logging.basicConfig(filename='output/nonlinear_{}.log'.format(date),level=logging.DEBUG)

dt = 5e-2
TOL = 1e-6
MAX_IT = 30
pnt = mesh(lx-d/2, 0, 0)
tval = np.arange(time_interval[0], time_interval[1]+dt, dt)


gfu_old = GridFunction(fes)

logging.info("""
TOL = {}
MAX_IT = {}
time values = {}
eval point = {}
angluar frequency = {}
""".format(TOL, MAX_IT, tval, (lx-d/2, 0, 0), omega))


values = [None]*int(1/dt)
# achtung [np.zeros(MAX_STEPS)]*3 geht nicht weil Referenz auf gleiches Objekt
plot_data = {'B': [np.zeros(len(tval)) for _ in range(3)], 
             'A': [np.zeros(len(tval)) for _ in range(3)],
             'E': [np.zeros(len(tval)) for _ in range(3)]} 

# mv to store basis in
mv = None

with TaskManager():
    
    
    res = gfu.vec.CreateVector()
#     w = gfu.vec.CreateVector()
    tmp = gfu.vec.CreateVector()

    tmp_mat = a.mat.CreateMatrix()
    
    # initial u 
    gfu.vec[:] = 0
    
    # TODO: alternative
    # only for preconditioner
    b = BilinearForm(fes)
    for mat in mur.keys():
        b += 1/(mu0*mur[mat])*curl(u)*curl(v)*dx(mat) + 1e-6*1/(mu0*mur[mat])*u*v*dx(mat)
    b += Variation(energy_dens(sqrt(1e-12+ curl(u)*curl(u))) * dx('plate'))
    b += sigma['plate']/dt*(u-gfu_old)*v*dx('plate')
    b += -j*v*dx('coil')
    c = Preconditioner(b, "bddc")
    b.AssembleLinearization(gfu.vec) # for preconditioner
    
    for step in range(len(tval)):
        
        print("\nImplicit Euler Iteration: {}\n".format(step))
        logging.info("\nImplicit Euler Iteration: {}\n".format(step))

        t.Set(tval[step])
        icd.Assemble()

        gfu_old.vec.data = gfu.vec
        
        err = 1
        it = 0
        
        ####### begin solve problem
        
        while err > TOL and it < MAX_IT:
            it += 1
            
            # TODO: implement control if energy is descending
#             e_old = b.Energy(gfu.vec)
            
            # newton step 1
            g.Apply(gfu.vec, res)
            res += a.mat*gfu.vec + proj*icd.vec + \
                1/dt*(cur.mat*gfu.vec - cur.mat*gfu_old.vec)
            
#             res.data = proj*tmp 
            
            # newton step 2
            g.AssembleLinearization(gfu.vec)
            tmp_mat = g.mat+a.mat + 1/dt*cur.mat


            # newton step 3
#             b.AssembleLinearization(gfu.vec) # for preconditioner
            solvers.CG(sol=tmp, rhs=res, mat=tmp_mat, pre=c.mat, printrates=False)

#             e_old = b.Energy(gfu.vec)

#             b.Apply(gfu.vec, res)

#             b.AssembleLinearization(gfu.vec)

#             solvers.CG(sol=w, rhs=res, mat=b.mat, pre=c.mat, printrates=False)

            err = abs(InnerProduct(tmp, res))
            
            
            print(" error {} in newton iteration {}".format(err, it))
            logging.info(" error {} in newton iteration {}".format(err, it))

            res.data = gfu.vec - tmp
            
#             e_new = g.Energy(tmp)
            
#             tau = 1
#             cnt = 1
#             while e_new > e_old:
#                 tau *= max(0.1, 1-0.1*cnt)
#                 cnt += 1
#                 tmp.data = gfu.vec - tau*w
#                 e_new = g.Energy(tmp)
#                 print(" new energy: {} with tau: {}".format( e_new, tau))

            gfu.vec.data = res
    
            
        index = int(step % (1/dt))
        if (values[index]):
            print("B({}): {}\nB({}-1): {}".format(t.Get(), values[index], t.Get(), curl(gfu)(pnt)))
            logging.info("B({}): {}\nB({}-1): {}".format(t.Get(), values[index], t.Get(), curl(gfu)(pnt)))

        else:
            print("B({}): {}".format(t.Get(), curl(gfu)(pnt)))
            logging.info("B({}): {}".format(t.Get(), curl(gfu)(pnt)))

        values[index] = curl(gfu)(pnt)
        
        for i in range(3):
            plot_data['B'][i][step] = curl(gfu)(pnt)[i]
            plot_data['A'][i][step] = gfu(pnt)[i]
            plot_data['E'][i][step] = ((gfu-gfu_old)/dt)(pnt)[i]

        ####### end solve problem
    
        if mv is None:
            mv = MultiVector(gfu.vec, 1)
        else:
            mv.AppendOrthogonalize(gfu.vec)
    
    

In [ ]:
fig = plt.figure()
axis = plt.axes()

print(tval, plot_data['B'][0])
# x_val = np.arange(0, MAX_STEPS*dt, dt)
# x_val = tval
plt.plot(tval, plot_data['B'][0], label='x')
plt.plot(tval, plot_data['B'][1], label='y')
plt.plot(tval, plot_data['B'][2], label='z')
plt.title("B(0.35,0,0)")
plt.legend()

In [ ]:
# save data
npmv = np.zeros((len(mv[0]), len(mv)))#, dtype=complex)
for j in range(len(mv)):
    npmv[:,j] = mv[j].FV().NumPy()
    
outputFile = 'output/mv_{}.data'.format(date)
fw = open(outputFile, 'wb')
pickle.dump(npmv, fw)
fw.close()

In [5]:
# load data
inputFile = 'output/mv.data'
fd = open(inputFile, 'rb')
npmv = pickle.load(fd)

V = MultiVector(npmv.shape[0],npmv.shape[1], False)

for j in range(len(V)):
    V[j].FV().NumPy()[:] = npmv[:,j]
    
print(len(V), len(V[0]))

5 614980


In [8]:
TOL = 1e-6
MAX_IT = 30
pnt = mesh(lx-d/2, 0, 0)
dt = 5e-3
tval = np.arange(time_interval[0], time_interval[1]+dt, dt)

logging.info("""
TOL = {}
MAX_IT = {}
time values = {}
eval point = {}
angluar frequency = {}
""".format(TOL, MAX_IT, tval, (lx-d/2, 0, 0), omega))


# build reduced matrices
mv = MultiVector(gfu.vec, len(V))

mv[:len(V)] = a.mat * V
# ared = InnerProduct(V,mv)

mv[:len(V)] = cur.mat * V
curred = InnerProduct(V,mv)

res = gfu.vec.CreateVector()

w = Vector(len(V))
w_old = Vector(len(V))
redres = Vector(len(V))


mv1 = MultiVector(a.vec, 1)
mvV = MultiVector(a.vec, V)

mat= Matrix(len(V), len(V))


values = [None]*int(1/dt)
# achtung [np.zeros(MAX_STEPS)]*3 geht nicht weil Referenz auf gleiches Objekt
red_plot_data = {'B': [np.zeros(len(tval)) for _ in range(3)], 
             'A': [np.zeros(len(tval)) for _ in range(3)],
             'E': [np.zeros(len(tval)) for _ in range(3)]} 

with TaskManager():
    
    
    # initial u 
    w[:] = 0 
    
    for step in range(len(tval)):
        
        print("\nImplicit Euler Iteration: {}\n".format(step))
        logging.info("\nImplicit Euler Iteration: {}\n".format(step))

        t.Set(tval[step])
        icd.Assemble()
        mv1[0] = proj*icd.vec
        icdred = InnerProduct(V, mv1)

        w_old[:] =  w
        
        err = 1
        it = 0
        
        ####### begin solve problem
        
        while err > TOL and it < MAX_IT:
            it += 1
            
            # TODO: implement control if energy is descending
#             e_old = b.Energy(gfu.vec)

            gfu.vec.data = V * w
            
            # newton step 1
            g.Apply(gfu.vec, res)
            
            mv1[0] = res.vec
            resred.data = InnerProduct(V, mv1)
            
            break
            res += a.mat*gfu.vec + proj*icd.vec + \
                1/dt*(cur.mat*gfu.vec - cur.mat*gfu_old.vec)
            
#             res.data = proj*tmp 
            
            # newton step 2
            g.AssembleLinearization(gfu.vec)
            tmp_mat = g.mat+a.mat + 1/dt*cur.mat


            # newton step 3
#             b.AssembleLinearization(gfu.vec) # for preconditioner
            solvers.CG(sol=tmp, rhs=res, mat=tmp_mat, pre=c.mat, printrates=False)

#             e_old = b.Energy(gfu.vec)

#             b.Apply(gfu.vec, res)

#             b.AssembleLinearization(gfu.vec)

#             solvers.CG(sol=w, rhs=res, mat=b.mat, pre=c.mat, printrates=False)

            err = abs(InnerProduct(tmp, res))
            
            
            print(" error {} in newton iteration {}".format(err, it))
            logging.info(" error {} in newton iteration {}".format(err, it))

            res.data = gfu.vec - tmp
            
#             e_new = g.Energy(tmp)
            
#             tau = 1
#             cnt = 1
#             while e_new > e_old:
#                 tau *= max(0.1, 1-0.1*cnt)
#                 cnt += 1
#                 tmp.data = gfu.vec - tau*w
#                 e_new = g.Energy(tmp)
#                 print(" new energy: {} with tau: {}".format( e_new, tau))

            gfu.vec.data = res
    
            
        index = int(step % (1/dt))
        if (values[index]):
            print("B({}): {}\nB({}-1): {}".format(t.Get(), values[index], t.Get(), curl(gfu)(pnt)))
            logging.info("B({}): {}\nB({}-1): {}".format(t.Get(), values[index], t.Get(), curl(gfu)(pnt)))

        else:
            print("B({}): {}".format(t.Get(), curl(gfu)(pnt)))
            logging.info("B({}): {}".format(t.Get(), curl(gfu)(pnt)))

        values[index] = curl(gfu)(pnt)
        
        for i in range(3):
            plot_data['B'][i][step] = curl(gfu)(pnt)[i]
            plot_data['A'][i][step] = gfu(pnt)[i]
            plot_data['E'][i][step] = ((gfu-gfu_old)/dt)(pnt)[i]

        ####### end solve problem
    
        if mv is None:
            mv = MultiVector(gfu.vec, 1)
        else:
            mv.AppendOrthogonalize(gfu.vec)
    
    
    

TypeError: object of type 'ngsolve.la.MultiVectorExpr' has no len()